<a href="https://colab.research.google.com/github/alloysArtifexLabs/IMAGE-AI/blob/image-classifier/Mental_Health_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries (uncomment if needed)
# !pip install tensorflow==2.8.0
# !pip install pandas numpy scikit-learn

# Import packages and display versions for confirmation
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
import datetime
import random

print("TensorFlow version:", tf.__version__)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)


TensorFlow version: 2.18.0
Pandas version: 2.2.2
NumPy version: 1.26.4


In [2]:
# Create a small labeled dataset for sentiment classification.
# Labels: 0 = Negative, 1 = Neutral, 2 = Positive
data = {
    'text': [
        "I feel anxious today",
        "I'm doing okay",
        "I'm really happy",
        "I am overwhelmed and stressed",
        "Everything is fine",
        "I love my life",
        "I feel sad and depressed",
        "Today was a tough day",
        "I am feeling excited about tomorrow",
        "I am not in a good mood"
    ],
    'label': [0, 1, 2, 0, 1, 2, 0, 0, 2, 0]
}

df = pd.DataFrame(data)
print("Sample Data:")
print(df.head())

# Split the data into training and test sets.
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define text preprocessing using TensorFlow's TextVectorization.
max_features = 1000     # Maximum vocabulary size
sequence_length = 20    # Maximum sequence length

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

# Adapt the vectorization layer on the training text.
vectorize_layer.adapt(train_df['text'].values)


Sample Data:
                            text  label
0           I feel anxious today      0
1                 I'm doing okay      1
2               I'm really happy      2
3  I am overwhelmed and stressed      0
4             Everything is fine      1


In [3]:
embedding_dim = 16  # Dimension for embedding

model = tf.keras.Sequential([
    vectorize_layer,  # Preprocessing layer (tokenizes and vectorizes text)
    tf.keras.layers.Embedding(max_features, embedding_dim, name="embedding"),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 sentiment classes
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization                   │ ?                           │     0 (unbuilt) │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Prepare training and testing data
x_train = train_df['text'].values
y_train = train_df['label'].values
x_test = test_df['text'].values
y_test = test_df['label'].values

# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print("\nTest Accuracy: {:.2f}%".format(accuracy * 100))


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 1.1086 - val_accuracy: 0.5000 - val_loss: 1.0908
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.2500 - loss: 1.1043 - val_accuracy: 0.5000 - val_loss: 1.0950
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.2500 - loss: 1.1003 - val_accuracy: 0.5000 - val_loss: 1.0987
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.2500 - loss: 1.0968 - val_accuracy: 0.5000 - val_loss: 1.1023
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.2500 - loss: 1.0948 - val_accuracy: 0.5000 - val_loss: 1.1043
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.3750 - loss: 1.0934 - val_accuracy: 0.0000e+00 - val_loss: 1.1060
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.3750 - loss: 1.0920 - val_accuracy: 0.0000e+00 - val_loss: 1.1076
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.7500 - loss: 1.0907 - val_accuracy: 0.0000e+00 - va

In [9]:
# --- Empathy & Resources Library ---

# Empathy statements for supportive responses.
empathy_statements = [
    "I'm here to listen.",
    "I understand that can be difficult.",
    "It sounds like you're going through a tough time.",
    "Your feelings are important."
]

# Breathing exercises and stress relief tips.
breathing_exercises = [
    "Try taking a slow deep breath in for 4 counts, hold for 4, and exhale for 4.",
    "Consider a 5-5-5 breathing technique: breathe in for 5 seconds, hold for 5, and exhale for 5."
]

# Mental health resources and hotlines.
mental_health_resources = [
    "National Suicide Prevention Lifeline: 988 (US only)",
    "Crisis Text Line: Text HOME to 741741 (US only)",
    "For UK: Samaritans - 116 123",
    "Visit https://www.mentalhealth.gov/ for more resources."
]

# Dictionary to track user mood logs (in a real application, consider persistent storage).
user_mood_logs = {}

# --- Helper Functions ---

def classify_sentiment(text):
    """Predict sentiment of the input text using the trained model."""
    prediction = model.predict([text])
    sentiment = tf.argmax(prediction, axis=1).numpy()[0]
    return sentiment

def get_supportive_response(sentiment):
    """
    Generate a supportive response based on the sentiment.
    0: Negative, 1: Neutral, 2: Positive
    """
    if sentiment == 0:
        response = random.choice(empathy_statements) + " It seems you're feeling down."
        response += "\n" + random.choice(breathing_exercises)
        response += "\nIf you're feeling extremely distressed, please consider reaching out to a professional. " + random.choice(mental_health_resources)
    elif sentiment == 1:
        response = "Thanks for sharing. It sounds like things are okay. Remember, I'm here if you need someone to talk to."
    elif sentiment == 2:
        response = "I'm glad to hear that you're feeling good! Keep up the positive vibes."
    else:
        response = "I'm sorry, I didn't quite understand that. Could you please tell me more?"
    return response

def record_mood(user_input, sentiment):
    """
    Ask the user if they'd like to record their mood.
    If yes, store the input text and sentiment along with a timestamp.
    """
    record = input("Would you like to record your mood? (yes/no): ").strip().lower()
    if record == 'yes':
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        user_mood_logs[timestamp] = {'text': user_input, 'sentiment': sentiment}
        print("Your mood has been recorded.")
    else:
        print("No problem, I won't record your mood.")

def conversation_loop():
    """
    Main conversation loop for the Mental Health & Well-Being Bot.

    Disclaimer: This bot is designed for supportive interactions only and does not replace professional help.

    To exit the conversation, type 'exit'.
    """
    print("Welcome to the Mental Health & Well-Being Bot.")
    print("Disclaimer: I am not a substitute for professional help. If you are in crisis, please seek immediate assistance.")
    print("Type 'exit' at any time to end the conversation.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Thank you for chatting. Take care!")
            break

        # Classify the sentiment of the user input.
        sentiment = classify_sentiment(user_input)
        # Get a supportive response based on the sentiment.
        response = get_supportive_response(sentiment)
        print("\nBot:", response)

        # Ask if the user would like to record their mood.
        record_mood(user_input, sentiment)

# Start the conversation loop.
conversation_loop()


Welcome to the Mental Health & Well-Being Bot.
Disclaimer: I am not a substitute for professional help. If you are in crisis, please seek immediate assistance.
Type 'exit' at any time to end the conversation.

You: exit
Thank you for chatting. Take care!
